In [9]:
# WARNING: This notebook is for use with Jupyter Lab ONLY.

# Hello!
# Please run this cell either with Shift + Enter
# Afterwards you will be prompted to enter an initial configuration file, please ensure that you know the path for your
# desired input file.
# Thank you!

'''
This section is for import statements and setting up run-time variables, such as the home directory as well as needed widget constructs.
'''
import ipywidgets as widgets
from ipywidgets import Box, Layout, HBox
%matplotlib ipympl
import sys
import os
import importlib
import MASTML
from MASTMLInitializer import ConfigFileConstructor
home = os.getcwd()
template_path = "" #This is used only if using the branch "Luke-Max-Experimental of the repository"
widgLayout = Layout(width = '45%', height = '40px',font_size='12px')
output = widgets.Output()

#This function is for arranging widgets in a vertical box
def VBox(*pargs, **kwargs):
    """Displays multiple widgets vertically using the flexible box model."""
    box = Box(*pargs, **kwargs)
    box.layout.display = 'flex'
    box.layout.flex_flow = 'column'
    box.layout.align_items = 'stretch'
    box.layout.border = 'solid'
    return box

'''
This section defines functions needed for outputting the widgets values to be saved to new configuration files.
'''
#This function turns the values of the widgets into an array to be printed to a text file
def arrayifyBox(Box):
    out = []
    for i in range(len(Box.children)):
        state = Box.children[i].get_state()
        if 'BoxModel' not in state['_model_name']: #child is a widget
            try:
                if state['description'] == "field": #chlid is a label of a field
                    temp = str(state['value']) + "="
                else: #child is a widget with a value
                    if state['_model_name'] == 'SelectMultipleModel':
                        tup = Box.children[i].value
                        temp = ' ' + ','.join(tup)
                    else:
                        temp = ' ' + str(Box.children[i].value)
                out.append(temp.rstrip())
            except: KeyError
        else: #child is a box
            out.append(arrayifyBox(Box.children[i])) #grab the widgets from the child
    return out

#This function exports the values of the widgets to a text file
def exportBox(Box, filename):
    arr = arrayifyBox(Box)
    file = open(filename, "w")
    for lines in arr:
        for line in lines:
            file.write(line)
        file.write("\n")
    file.close()

'''
This section defines the functions needed to import a configuration files and generate the widgets to be displayed to the user.
'''
#This function imports the file and puts the lines of text into an array
def importArray(filename):
    file = open (filename, "r")
    arr = []
    for line in file.readlines():
        if (line.rstrip() != '' and line.strip()[0] != '#'): #line is not empty or a comment
            arr.append(line.rstrip())
    file.close()
    return arr

#This function handles implicit typing for the widgets based on the values specified in the configuration file
def typeWidgets(arr):
    types = []
    for i in range(len(arr)):
        split = arr[i].split('=')
        if(len(split)==1): #element is a label
            types.append("Label")
        else:
            val = split[1].strip() #fetch the value
            try:
                x = float(val)
                if x % 1 == 0:
                    types.append("Int")
                else:
                    types.append("Float")
            except ValueError:
                if val == "True" or val == "False":
                    types.append("Boolean")
                else:
                    types.append("String")
    return types


#This function gets the test possibilities from the template dict that is created by MASTML Initializer
def getTests(arr,template):
    possibles = []
    for i in range(len(arr)):
        desc = arr[i].split('=')[0].strip()
        if(desc[0]=='[' and desc[1]=='['): #if a double header
            desc=desc.strip('[')
            desc=desc.strip(']')
            possibles.append(desc) #list it as a possible
    tests=[]
    for j in range(len(possibles)):
        testCase=possibles[j].split('_')[0] #get the test name
        if (testCase in template['Models and Tests to Run']['test_cases']):
            tests.append(possibles[j])
    return tests

#This function generates the list of acceptable field names from Max and Luke's template
def generateWhiteList():
    fields = []
    if (template_path != ""):
        os.chdir(template_path)
        file = open('template.conf','r')
        template = file.readlines()
        fields = []
        for i in range(len(template)):
            text = template[i].strip()
            if(text != '' and text[0] != '#'): #line is not a comment
                fields.append(text.split('#')[0].split('=')[0].strip().strip('[').strip(']'))
    return fields
 
#This function generates the widgets after the configuration file has been made into an array
def genWidgets(arr,template):
    #set up types and box
    types = typeWidgets(arr)
    box = []
    tests = getTests(arr,template)
    whiteList = generateWhiteList()
    for i in range(len(types)):
        desc = arr[i].split('#')[0].split('=')[0].strip()
        desc = desc.strip("[")
        desc = desc.strip("]")
        found = True
        if(desc not in template.keys() and desc not in whiteList):
            found = False
            for k in template.keys():
                if(desc in template[k].keys()):
                    found = True
                else:
                    try:
                        for l in template[k].keys():
                            if(desc in template[k][l].keys() or desc == "Initial"):
                                found = True
                    except: AttributeError
            if not found:
                print("Possible misspelled field: " + desc + " not recognized")
        #create appropriate widget
        if types[i] == 'String':
            widg = makeStringWidget(arr[i],template,tests)
        elif types[i] == 'Float':
            widg = makeFloatWidget(arr[i])
        elif types[i] == 'Int':
            widg = makeIntWidget(arr[i])
        elif types[i] == 'Label':
            widg = widgets.Label(value=desc,layout = Layout(align = 'center',height = '60px',font_size='24px'))
        else: #type is a bool
            widg = makeBoolWidget(arr[i])
        #add widget to box
        box.append(widg)
    return(VBox(box))

#Handles generating the widgets by calling "genWidgets" after an input file is specified
def loadConf(filename):
    arr = importArray(filename)
    template = ConfigFileConstructor(filename).get_config_template()
    return genWidgets(arr,template)

'''
This sub-section handles creating different types of widgets based on the value type
'''
#Makes string widgets, and handles determining if a dropdown should be used
def makeStringWidget(elem,template,tests):
    split = elem.split('=')
    desc = split[0]
    val = split[1]
    if(desc.strip() == 'feature_selection_algorithm'):
            opts = template['Feature Selection']['feature_selection_algorithm']
            widg = makeDropdown(desc,val,opts)
    elif(desc.strip() == 'models'):
            opts = template['Models and Tests to Run']['models']
            widg = makeDropdown(desc,val,opts)
    elif(desc.strip() == 'test_cases'):
            opts = tests
            widg = makeSelectMutli(desc,[val],opts)
    elif(desc.strip() == 'model_to_use_for_learning_curve'):
            opts = template['Feature Selection']['model_to_use_for_learning_curve']
            widg = makeDropdown(desc,val,opts)
    elif(desc.strip() == 'scoring_metric'):
            opts = template['Feature Selection']['scoring_metric']
            widg = makeDropdown(desc,val,opts)
    else:
        widg = HBox([widgets.Label(value=desc, description = 'field', layout = widgLayout),
                 widgets.Text(
            value=val,
            placeholder = '',
            description = '',
            layout = widgLayout,
            disabled = False
        )])
    return widg

#Makes a float widget
def makeFloatWidget(elem):
    split = elem.split('=')
    desc = split[0]
    val = split[1]
    widg = HBox([widgets.Label(value=desc, description = 'field',layout = widgLayout),
                 widgets.FloatText(
            value=val,
            placeholder = '',
            description = '',
            layout = widgLayout,
            disabled = False
    )])
    return widg

#Makes an int widget
def makeIntWidget(elem):
    split = elem.split('=')
    desc = split[0]
    val = split[1]
    widg = HBox([widgets.Label(value=desc, description = 'field',layout = widgLayout),
                 widgets.IntText(
            value=val,
            placeholder = '',
            description = '',
            layout = widgLayout,
            disabled = False
    )])
    return widg

#Makes a widgets for true or false values
def makeBoolWidget(elem):
    split = elem.split('=')
    desc = split[0]
    val = split[1].strip()
    widg = HBox([widgets.Label(value=desc, description = 'field',layout = widgLayout),
                 widgets.Dropdown(
        options=['True','False'],
        value = val,
        description = '',
        layout = widgLayout,
        disabled=False
    )])
    return widg

#Makes a dropdown widget for fields with a discrete number of values
def makeDropdown(desc,val,opts):
    widg = HBox([widgets.Label(value=desc,description = 'field',layout = widgLayout),
                    widgets.Dropdown(
                options = opts,
                value = val.strip(),
                description = '',
                layout=widgLayout,
                disabled = False
            )])
    return widg

#Makes a SelectMulti widget for fields with a discrete number of values where values can be a list
def makeSelectMutli(desc,val,opts):
    for i in range(len(val)):
        val[i] = val[i].strip()
    widg = HBox([widgets.Label(value=desc,description = 'field',layout = widgLayout),
                    widgets.SelectMultiple(
                options = opts,
                value = val,
                description = '',
                width = '45%', 
                height = '100px',
                font_size='12px',
                disabled = False
            )])
    return widg

'''
This section sets up the boxes needed for initial IO such as loading, saving, and running configuration files
'''
#Lines for the runBox
runButton = widgets.Button(description='Run Test')
inFile = widgets.Text(placeholder = 'enter input file path',val='',disabled = False)
runBox = HBox([inFile,runButton])
#Lines for the saveBox
saveButton = widgets.Button(description='Save to File')
outFile = widgets.Text(placeholder = 'enter output file path', val='', disabled = False)
saveBox= HBox([outFile,saveButton])
#Lines for the loadBox
loadButton = widgets.Button(description='Load config file')
initConf = widgets.Text(placeholder = 'enter initial configuration file path', val='', disabled = False)
loadBox = HBox([initConf,loadButton])

'''
This section sets up the on_clicked() events for each of the buttons
'''
@output.capture()
#Handles running MASTML given a specified input file
def on_runButton_clicked(b):
    from IPython.core.display import display, HTML
    #display(HTML("<style>.container { width:100% !important; }</style>"))
    conFile = runBox.children[0].get_state()['value']
    cwd = os.getcwd()
    in_path = os.path.dirname(os.path.realpath(conFile))
    os.chdir(in_path)
    fileName = conFile.split('/')[-1]
    mastml = MASTML.MASTMLDriver(configfile=fileName)
    mastml.run_MASTML()
    os.chdir(cwd)

@output.capture()
#Handles saving the widget values to a specified file path
def on_saveButton_clicked(b):
    conFile = saveBox.children[0].get_state()['value']
    allWidgets = IOBox.children[1]
    exportBox(allWidgets, conFile)
    print("File saved!")

@output.capture()
#Handles loading the widget values from a specified conf file
def on_loadButton_clicked(b):
    os.chdir(home)
    conFile = loadBox.children[0].get_state()['value']
    if(conFile != ''):
        from IPython.display import display
        global IOBox
        IOBox.close()
        IOBox = VBox([loadBox,saveBox,runBox])
        allWidgets = loadConf(conFile)
        IOBox.children = (IOBox.children[0],allWidgets,IOBox.children[1],IOBox.children[2])
        display(IOBox)

#These lines bind the on_click events
runButton.on_click(on_runButton_clicked)
saveButton.on_click(on_saveButton_clicked)
loadButton.on_click(on_loadButton_clicked)

'''
This section displays the initial cell output
'''
IOBox = VBox([loadBox,runBox])
display(IOBox)
display(output)

Box(children=(HBox(children=(Text(value='', placeholder='enter initial configuration file path'), Button(descr…

Output()

In [25]:
template_path = "./"

def generateList():
    if template_path != ""
        os.chdir(template_path)
        file = open('template.conf','r')
        template = file.readlines()
        fields = []
        for i in range(len(template)):
            text = template[i].strip()
            if(text != '' and text[0] != '#'): #line is not a comment
                fields.append(text.split('#')[0].split('=')[0].strip())
        print(fields)

In [26]:
generateList()

['[GeneralSetup]', 'input_features', 'target_feature', 'validation_column', '[FeatureNormalization]', '[[MeanStdevScaler]]', '[[MinMaxScaler]]', '[FeatureGeneration]', '[[Magpie]]', 'api_key', '[[MaterialsProject]]', 'api_key', '[[Citrine]]', 'api_key', '[[PolynomialFeatures]]', '[[ContainsElement]]', 'composition_feature', 'element', 'new_name', '[FeatureSelection]', '[[RemoveConstantFeatures]]', '[[PrincipleComponentAnalysis]]', '[SelectKBest]', 'k', 'scoring', '[[VarianceThreshold]]', 'threshold', '[[SelectPercentile]]', 'percentile', 'scoring', '[[SelectFpr]]', '[[SelectFdr]]', '[[SelectFwe]]', '[[GenericUnivariateSelect]]', '[[RFE]]', '[DataSplits]', '[[NoSplit]]', '[[JustEachGroup]]', 'grouping_features', '[[WithoutEachGroup]]', 'grouping_feature', 'plots', '[[KFold]]', 'k', '[[LeaveOutPercent]]', 'percentage', '[[LeaveOneOut]]', '[Models]', '[[SVR]]', 'C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', '[[Lasso]]', 'alpha', 'copy_X', 'fit